# 🧠 Módulo 4: Conhecimento e RAG - Transformando Agentes em Especialistas

## Bem-vindos ao mundo do conhecimento especializado!

Tá, mas o que é esse tal de **Knowledge** no Agno? Imagina que você tem um agente super inteligente, mas que não sabe nada sobre o seu negócio específico. É como contratar um PhD em física para trabalhar numa loja de roupas - ele é super inteligente, mas não sabe nada sobre moda!

![](https://s3.us-east-1.amazonaws.com/turing.education/books/imagens/agno-2.0-framework--modulo-04_img_01.png)

Nos módulos anteriores, aprendemos:
- **Módulo 1**: Como criar agentes básicos
- **Módulo 2**: Como configurar modelos e instruções
- **Módulo 3**: Como dar ferramentas aos agentes

Agora vamos dar **conhecimento especializado** aos nossos agentes!

## O que vamos aprender hoje:
1. 🎯 O que é Knowledge e Agentic RAG
2. 🔍 Vector Databases na prática
3. 🧮 Embeddings e como funcionam
4. 🤖 Implementando RAG com Agno
5. 🚀 Casos práticos avançados

# 🎯 O que é Knowledge e por que precisamos?

Vamos começar com uma analogia simples: **Knowledge** é como dar uma biblioteca especializada para o seu agente!

## O problema sem Knowledge:
- Agente só sabe o que foi treinado (até uma data de corte)
- Não conhece informações específicas da sua empresa
- Pode "alucinar" informações sobre tópicos que não domina

## A solução com Knowledge:
- **Dynamic Few-Shot Learning**: O agente busca exemplos relevantes na hora
- **Agentic RAG**: Busca informações quando precisa (não sempre)
- **Conhecimento atualizado**: Sempre com as informações mais recentes

### Analogia do Bibliotecário Inteligente 📚

Imagine que seu agente é um bibliotecário super inteligente:
- **Sem Knowledge**: Ele só responde com o que decorou
- **Com Knowledge**: Ele pesquisa nos livros certos antes de responder

**Dica!** No Agno, o Knowledge é usado de forma **agentic** - o agente decide quando precisa buscar informações!

# 🔧 Setup Inicial - Preparando o Ambiente

Bora preparar nosso ambiente para trabalhar com Knowledge!

In [ ]:
# Instalando as dependências necessárias
!pip install -q agno google-generativeai qdrant-client sentence-transformers

print("📦 Bibliotecas instaladas com sucesso!")
print("Agora vamos configurar nossa API key do Google...")

In [ ]:
import os
from getpass import getpass

# Configurando a API Key do Google (gratuita!)
if 'GOOGLE_API_KEY' not in os.environ:
    os.environ['GOOGLE_API_KEY'] = getpass('🔑 Cole sua API Key do Google AI Studio: ')
    
print("✅ API Key configurada!")
print("💡 Dica: Pega sua chave gratuita em https://aistudio.google.com/")

In [ ]:
# Imports básicos para nosso módulo
import asyncio
from agno.agent import Agent
from agno.models.google import Gemini
from agno.knowledge.knowledge import Knowledge
from agno.vectordb.qdrant import Qdrant
from agno.embedder.google import GoogleEmbedder

print("📚 Importações realizadas!")
print("Agora temos tudo que precisamos para criar agentes inteligentes!")

# 🧮 Entendendo Embeddings - A Matemática por Trás da Magia

Antes de partir para a prática, vamos entender **como funciona a mágica** dos embeddings!

## O que são Embeddings?

Embeddings são **representações numéricas** de texto. É como transformar palavras em coordenadas matemáticas!

### Analogia do GPS 🗺️
- **Texto**: "Padaria da esquina"
- **Embedding**: [-23.5505, -46.6333] (coordenadas)
- **Similaridade**: Distância entre pontos no espaço

### Como funciona na prática:
1. **Texto** → **Embedding** (vetor de números)
2. Textos similares ficam **próximos** no espaço matemático
3. Busca por **similaridade** usando distância vetorial

**Matemática por trás:**
- Cosseno entre vetores: $\cos(\theta) = \frac{A \cdot B}{||A|| \cdot ||B||}$
- Distância euclidiana: $d = \sqrt{\sum_{i=1}^{n}(a_i - b_i)^2}$

In [ ]:
# Vamos ver embeddings na prática!
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Simulando embeddings de diferentes textos
# (Na realidade, embeddings têm centenas ou milhares de dimensões)
textos = [
    "cachorro", "gato", "hamster",  # animais
    "carro", "moto", "bicicleta",   # veículos
    "pizza", "hambúrguer", "lasanha" # comidas
]

# Embeddings simulados (3 grupos distintos)
embeddings_animais = np.random.normal([1, 1], 0.2, (3, 2))
embeddings_veiculos = np.random.normal([3, 1], 0.2, (3, 2))
embeddings_comidas = np.random.normal([2, 3], 0.2, (3, 2))

# Juntando tudo
embeddings = np.vstack([embeddings_animais, embeddings_veiculos, embeddings_comidas])
cores = ['red', 'red', 'red', 'blue', 'blue', 'blue', 'green', 'green', 'green']

# Visualizando
plt.figure(figsize=(10, 8))
plt.scatter(embeddings[:, 0], embeddings[:, 1], c=cores, s=100)

for i, texto in enumerate(textos):
    plt.annotate(texto, (embeddings[i, 0], embeddings[i, 1]), 
                xytext=(5, 5), textcoords='offset points', fontsize=12)

plt.title('🧮 Visualização de Embeddings - Textos Similares Ficam Próximos', fontsize=14)
plt.xlabel('Dimensão 1')
plt.ylabel('Dimensão 2')
plt.grid(True, alpha=0.3)
plt.legend(['Animais', 'Veículos', 'Comidas'], loc='upper right')
plt.show()

print("🎯 Viu como textos similares ficam agrupados?")
print("É assim que o agente 'entende' o significado dos textos!")

# 🗄️ Vector Databases - O Cérebro do Sistema

Agora que entendemos embeddings, vamos falar sobre **Vector Databases**!

## O que são Vector Databases?

São bancos de dados **especializados** em armazenar e buscar vetores (embeddings) de forma super eficiente!

### Analogia da Biblioteca Mágica 📖
- **Banco tradicional**: "Me dê o livro com ID 123"
- **Vector Database**: "Me dê os 5 livros mais similares a este tema"

### Como funciona o processo:
1. **Chunking**: Quebra documentos em pedaços menores
2. **Embedding**: Converte cada chunk em vetor
3. **Indexing**: Armazena de forma otimizada
4. **Search**: Busca por similaridade vetorial

**Dica!** No Agno, usamos Qdrant como vector database - é rápido e fácil de usar!

In [ ]:
# Vamos criar nossa primeira base de conhecimento!
# Simulando um banco de dados vetorial simples

class VectorDBSimples:
    def __init__(self):
        self.documentos = []
        self.embeddings = []
    
    def adicionar_documento(self, texto, embedding):
        self.documentos.append(texto)
        self.embeddings.append(embedding)
        print(f"📄 Documento adicionado: '{texto[:50]}...'")
    
    def buscar_similares(self, query_embedding, k=3):
        """Busca os k documentos mais similares"""
        similaridades = []
        
        for i, doc_embedding in enumerate(self.embeddings):
            # Calculando similaridade (produto escalar simplificado)
            similaridade = np.dot(query_embedding, doc_embedding)
            similaridades.append((similaridade, i))
        
        # Ordenando por similaridade (maior primeiro)
        similaridades.sort(reverse=True)
        
        resultados = []
        for sim, idx in similaridades[:k]:
            resultados.append({
                'documento': self.documentos[idx],
                'similaridade': sim,
                'indice': idx
            })
        
        return resultados

# Criando nossa base vetorial
vector_db = VectorDBSimples()
print("🗄️ Vector Database criado com sucesso!")

In [ ]:
# Adicionando conhecimento sobre IA à nossa base
conhecimentos_ia = [
    ("Machine Learning é um subcampo da inteligência artificial que permite aos computadores aprender sem serem programados explicitamente", 
     np.array([0.8, 0.6, 0.4, 0.9])),
    
    ("Deep Learning usa redes neurais com múltiplas camadas para reconhecer padrões complexos em dados", 
     np.array([0.9, 0.7, 0.3, 0.8])),
    
    ("Processamento de Linguagem Natural (NLP) permite que computadores entendam e processem linguagem humana", 
     np.array([0.7, 0.8, 0.5, 0.6])),
    
    ("Computer Vision é a área que permite aos computadores interpretar e analisar imagens e vídeos", 
     np.array([0.6, 0.4, 0.9, 0.7])),
    
    ("Reinforcement Learning é um tipo de aprendizado onde agentes aprendem através de recompensas e punições", 
     np.array([0.8, 0.5, 0.2, 0.9]))
]

# Adicionando à nossa base
print("📚 Adicionando conhecimentos à base vetorial...")
for texto, embedding in conhecimentos_ia:
    vector_db.adicionar_documento(texto, embedding)

print(f"\n✅ Base de conhecimento criada com {len(vector_db.documentos)} documentos!")

In [ ]:
# Testando a busca por similaridade
print("🔍 Testando busca por similaridade...\n")

# Query sobre redes neurais
query = "redes neurais e aprendizado profundo"
query_embedding = np.array([0.9, 0.8, 0.3, 0.7])  # Embedding simulado da query

print(f"🤔 Pergunta: '{query}'")
print("\n📋 Documentos mais relevantes:")

resultados = vector_db.buscar_similares(query_embedding, k=3)

for i, resultado in enumerate(resultados, 1):
    print(f"\n{i}. 📄 Similaridade: {resultado['similaridade']:.3f}")
    print(f"   📝 Documento: {resultado['documento']}")

print("\n🎯 Viu como encontrou os documentos mais relevantes?")
print("É assim que funciona um sistema RAG básico!")

# 🚀 Implementando RAG Real com Agno

Agora vamos partir para o **mundo real**! Vamos criar um sistema RAG completo usando Agno.

## O que é Agentic RAG?

**RAG Tradicional**: Sempre busca na base de conhecimento
**Agentic RAG**: O agente **decide quando** buscar conhecimento

### Vantagens do Agentic RAG:
- ⚡ **Mais eficiente**: Só busca quando necessário
- 🎯 **Mais preciso**: Agente sabe quando precisa de informação externa
- 🧠 **Mais inteligente**: Combina conhecimento interno e externo

**Dica!** No Agno, você só precisa adicionar `search_knowledge=True` e pronto!

In [ ]:
# Criando nossa primeira base de conhecimento real com Agno
import tempfile
import os

# Vamos usar um vector database em memória por simplicidade
from agno.vectordb.pgvector import PgVector
from agno.vectordb.qdrant import Qdrant

# Para este exemplo, vamos simular com dados locais
# Na prática, você usaria Qdrant Cloud ou local

print("🏗️ Configurando base de conhecimento...")

# Criando embedder do Google (gratuito!)
embedder = GoogleEmbedder(
    model="models/text-embedding-004",
    api_key=os.environ['GOOGLE_API_KEY']
)

print("✅ Embedder configurado!")
print("💡 Using Google's text-embedding-004 model (gratuito!)")

In [ ]:
# Para este exemplo, vamos usar uma abordagem simplificada
# Criando conhecimento diretamente com texto

# Simulando documentos sobre tecnologia brasileira
documentos_tech_brasil = [
    "O Brasil tem um ecossistema de startups em crescimento, com destaque para fintechs como Nubank, PagSeguro e Stone.",
    "São Paulo é considerado o principal hub de tecnologia do Brasil, concentrando grande parte das startups e empresas de tech.",
    "O Recife é conhecido como o Porto Digital, um dos principais polos de tecnologia do Nordeste brasileiro.",
    "Belo Horizonte tem se destacado como um importante centro de desenvolvimento de software e inovação tecnológica.",
    "O Brasil é líder mundial em adoção de PIX, sistema de pagamentos instantâneos criado pelo Banco Central.",
    "Empresas brasileiras como Movile, Locaweb e Totvs são referências no mercado de tecnologia nacional.",
    "O mercado brasileiro de e-commerce cresceu significativamente, com players como Mercado Livre, B2W e Via Varejo.",
    "O agronegócio brasileiro utiliza tecnologias avançadas como agricultura de precisão, drones e IoT."
]

print("📚 Documentos sobre tech brasileira preparados!")
print(f"Total de documentos: {len(documentos_tech_brasil)}")

In [ ]:
# Vamos criar um agente com conhecimento!
# Primeiro, criamos o modelo
modelo = Gemini(
    id="gemini-2.0-flash",
    api_key=os.environ['GOOGLE_API_KEY']
)

print("🤖 Modelo Gemini configurado!")

# Agora vamos criar um agente especialista em tech brasileiro
agente_tech_brasil = Agent(
    name="Especialista Tech Brasil",
    model=modelo,
    description="Você é um especialista em tecnologia brasileira e startups nacionais.",
    instructions=[
        "Use informações específicas sobre o ecossistema tech brasileiro",
        "Sempre que possível, cite exemplos de empresas brasileiras",
        "Seja preciso com dados sobre o mercado nacional"
    ],
    markdown=True
)

print("👨‍💻 Agente especialista criado!")
print("Agora vamos testá-lo sem conhecimento específico...")

In [ ]:
# Testando o agente SEM base de conhecimento
print("🧪 TESTE 1: Agente SEM base de conhecimento")
print("=" * 50)

resposta_sem_conhecimento = agente_tech_brasil.run(
    "Me conte sobre o ecossistema de fintechs no Brasil e cite 3 exemplos específicos com detalhes sobre cada uma."
)

print("🤖 Resposta sem conhecimento específico:")
print(resposta_sem_conhecimento.content)
print("\n" + "="*50)
print("📝 Observação: O agente respondeu com conhecimento geral,")
print("mas pode não ter informações específicas e atualizadas.")

## 📊 Visualizando o Processo RAG

Vamos visualizar como funciona o processo de Retrieval-Augmented Generation:

In [ ]:
# Visualizando o processo RAG
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.patches import FancyBboxPatch

fig, ax = plt.subplots(1, 1, figsize=(14, 10))
ax.set_xlim(0, 10)
ax.set_ylim(0, 10)
ax.axis('off')

# Título
ax.text(5, 9.5, '🚀 Processo Agentic RAG com Agno', 
        fontsize=18, ha='center', weight='bold')

# Passo 1: Pergunta do usuário
box1 = FancyBboxPatch((0.5, 7.5), 2, 1, 
                      boxstyle="round,pad=0.1", 
                      facecolor='lightblue', edgecolor='blue')
ax.add_patch(box1)
ax.text(1.5, 8, '👤 Pergunta\ndo Usuário', ha='center', va='center', fontsize=10, weight='bold')

# Passo 2: Agente decide
box2 = FancyBboxPatch((3.5, 7.5), 2, 1, 
                      boxstyle="round,pad=0.1", 
                      facecolor='lightgreen', edgecolor='green')
ax.add_patch(box2)
ax.text(4.5, 8, '🤖 Agente\nDecide', ha='center', va='center', fontsize=10, weight='bold')

# Passo 3: Busca no conhecimento
box3 = FancyBboxPatch((6.5, 7.5), 2, 1, 
                      boxstyle="round,pad=0.1", 
                      facecolor='lightyellow', edgecolor='orange')
ax.add_patch(box3)
ax.text(7.5, 8, '🔍 Busca\nConhecimento', ha='center', va='center', fontsize=10, weight='bold')

# Base de conhecimento
box4 = FancyBboxPatch((1, 5), 3, 1.5, 
                      boxstyle="round,pad=0.1", 
                      facecolor='lightcoral', edgecolor='red')
ax.add_patch(box4)
ax.text(2.5, 5.75, '🗄️ Vector Database\n(Documentos + Embeddings)', 
        ha='center', va='center', fontsize=10, weight='bold')

# Contexto aumentado
box5 = FancyBboxPatch((5.5, 5), 3, 1.5, 
                      boxstyle="round,pad=0.1", 
                      facecolor='lightviolet', edgecolor='purple')
ax.add_patch(box5)
ax.text(7, 5.75, '📋 Contexto\nAumentado', ha='center', va='center', fontsize=10, weight='bold')

# Resposta final
box6 = FancyBboxPatch((3.5, 2.5), 3, 1.5, 
                      boxstyle="round,pad=0.1", 
                      facecolor='lightsteelblue', edgecolor='navy')
ax.add_patch(box6)
ax.text(5, 3.25, '💬 Resposta\nEnriquecida', ha='center', va='center', fontsize=12, weight='bold')

# Setas
arrows = [
    ((2.5, 8), (3.5, 8)),  # usuário -> agente
    ((5.5, 8), (6.5, 8)),  # agente -> busca
    ((7.5, 7.5), (7, 6.5)),  # busca -> contexto
    ((2.5, 6.5), (6, 6.5)),  # knowledge -> contexto
    ((6.5, 5), (5.5, 4))   # contexto -> resposta
]

for start, end in arrows:
    ax.annotate('', xy=end, xytext=start,
                arrowprops=dict(arrowstyle='->', lw=2, color='black'))

# Legenda
ax.text(5, 1, '✨ O agente DECIDE quando buscar conhecimento adicional!', 
        ha='center', fontsize=12, style='italic', 
        bbox=dict(boxstyle="round,pad=0.3", facecolor='lightyellow'))

plt.tight_layout()
plt.show()

print("🎯 Processo completo de Agentic RAG visualizado!")

# 🛠️ Implementação Completa com Knowledge

Agora vamos implementar um sistema RAG completo usando as ferramentas do Agno!

In [ ]:
# Implementando RAG simplificado (sem vector database externo)
# Vamos simular o processo de conhecimento

class ConhecimentoTechBrasil:
    def __init__(self):
        self.base_conhecimento = {
            "fintechs": {
                "nubank": "Nubank é a maior fintech da América Latina, com mais de 80 milhões de clientes. Fundada em 2013 por David Vélez, oferece conta digital, cartão de crédito sem anuidade e investimentos.",
                "pagseguro": "PagSeguro é uma das maiores empresas de pagamentos do Brasil, oferecendo soluções para e-commerce, maquininhas de cartão e serviços financeiros para PMEs.",
                "stone": "Stone é uma fintech brasileira especializada em soluções de pagamento, conhecida por suas maquininhas e serviços bancários para pequenas e médias empresas."
            },
            "startups": {
                "mercadolivre": "Mercado Livre é o maior marketplace da América Latina, fundado em 1999, hoje avaliado em bilhões de dólares e presente em 18 países.",
                "ifood": "iFood é o maior aplicativo de delivery de comida do Brasil, conectando restaurantes e consumidores em mais de 1000 cidades.",
                "99": "99 é uma startup brasileira de mobilidade urbana, adquirida pela Didi Chuxing, oferecendo corridas de carro, moto e outros serviços."
            },
            "hubs": {
                "sao_paulo": "São Paulo concentra 70% das startups brasileiras, com ecossistema maduro de investidores, aceleradoras e talentos tech.",
                "recife": "Porto Digital no Recife é o maior parque tecnológico do Brasil em faturamento, com mais de 350 empresas e 15 mil empregos.",
                "florianopolis": "Florianópolis é conhecida como Vale do Silício brasileiro, forte em software e inovação tecnológica."
            }
        }
    
    def buscar_informacao(self, query):
        """Simula busca de informações relevantes"""
        query_lower = query.lower()
        resultados = []
        
        for categoria, items in self.base_conhecimento.items():
            for chave, valor in items.items():
                if any(palavra in query_lower for palavra in [chave, categoria]):
                    resultados.append(valor)
        
        return resultados[:3]  # Retorna top 3 resultados

# Criando nossa base de conhecimento
conhecimento_brasil = ConhecimentoTechBrasil()
print("📚 Base de conhecimento sobre tech brasileira criada!")

In [ ]:
# Função para enriquecer o contexto do agente
def criar_agente_com_conhecimento(query):
    """Cria um agente com contexto enriquecido baseado na query"""
    
    # Busca informações relevantes
    informacoes_relevantes = conhecimento_brasil.buscar_informacao(query)
    
    # Cria contexto adicional
    contexto_adicional = "\n\nInformações específicas disponíveis:\n"
    for i, info in enumerate(informacoes_relevantes, 1):
        contexto_adicional += f"{i}. {info}\n"
    
    # Cria agente com conhecimento enriquecido
    agente_enriquecido = Agent(
        name="Especialista Tech Brasil PRO",
        model=modelo,
        description=f"Você é um especialista em tecnologia brasileira com acesso a informações específicas e atualizadas.{contexto_adicional}",
        instructions=[
            "Use SEMPRE as informações específicas fornecidas acima",
            "Cite dados concretos e exemplos das empresas mencionadas",
            "Se não tiver informação específica, deixe claro",
            "Seja preciso com números e fatos sobre o mercado brasileiro"
        ],
        markdown=True
    )
    
    return agente_enriquecido, informacoes_relevantes

print("🎯 Função de enriquecimento de contexto criada!")
print("Agora vamos testar com a mesma pergunta...")

In [ ]:
# Testando o agente COM base de conhecimento
print("🧪 TESTE 2: Agente COM base de conhecimento")
print("=" * 60)

query_teste = "Me conte sobre o ecossistema de fintechs no Brasil e cite 3 exemplos específicos com detalhes sobre cada uma."

# Criando agente enriquecido
agente_com_conhecimento, infos_usadas = criar_agente_com_conhecimento(query_teste)

print("🔍 Informações encontradas na base de conhecimento:")
for i, info in enumerate(infos_usadas, 1):
    print(f"{i}. {info[:100]}...")

print("\n" + "-"*60)
print("🤖 Gerando resposta enriquecida...\n")

resposta_com_conhecimento = agente_com_conhecimento.run(query_teste)

print("✨ Resposta com conhecimento específico:")
print(resposta_com_conhecimento.content)

print("\n" + "="*60)
print("🎯 Comparação:")
print("✅ COM conhecimento: Informações específicas, dados concretos, exemplos reais")
print("❌ SEM conhecimento: Informações gerais, pode conter imprecisões")

# 📈 Métricas e Avaliação do Sistema RAG

Como saber se nosso sistema RAG está funcionando bem? Vamos implementar algumas métricas!

In [ ]:
# Sistema de avaliação do RAG
import time
from datetime import datetime

class AvaliadorRAG:
    def __init__(self):
        self.metricas = {
            'consultas_realizadas': 0,
            'tempo_medio_resposta': 0,
            'documentos_encontrados': [],
            'historico_consultas': []
        }
    
    def avaliar_consulta(self, query, agente, conhecimento_obj):
        """Avalia uma consulta RAG"""
        inicio = time.time()
        
        # Busca documentos relevantes
        documentos = conhecimento_obj.buscar_informacao(query)
        
        # Cria agente enriquecido
        agente_enriquecido, _ = criar_agente_com_conhecimento(query)
        
        # Gera resposta
        resposta = agente_enriquecido.run(query)
        
        fim = time.time()
        tempo_resposta = fim - inicio
        
        # Atualiza métricas
        self.metricas['consultas_realizadas'] += 1
        self.metricas['documentos_encontrados'].append(len(documentos))
        
        # Calcula tempo médio
        if self.metricas['tempo_medio_resposta'] == 0:
            self.metricas['tempo_medio_resposta'] = tempo_resposta
        else:
            self.metricas['tempo_medio_resposta'] = (
                self.metricas['tempo_medio_resposta'] + tempo_resposta
            ) / 2
        
        # Salva no histórico
        self.metricas['historico_consultas'].append({
            'timestamp': datetime.now().strftime('%H:%M:%S'),
            'query': query[:50] + '...' if len(query) > 50 else query,
            'documentos_encontrados': len(documentos),
            'tempo_resposta': round(tempo_resposta, 2),
            'qualidade_estimada': self._estimar_qualidade(documentos, resposta.content)
        })
        
        return resposta, {
            'documentos_encontrados': len(documentos),
            'tempo_resposta': round(tempo_resposta, 2),
            'documentos': documentos
        }
    
    def _estimar_qualidade(self, documentos, resposta):
        """Estima qualidade da resposta (método simplificado)"""
        if not documentos:
            return "Baixa"
        
        # Verifica se a resposta usa informações dos documentos
        palavras_docs = set()
        for doc in documentos:
            palavras_docs.update(doc.lower().split())
        
        palavras_resposta = set(resposta.lower().split())
        intersecao = len(palavras_docs.intersection(palavras_resposta))
        
        if intersecao > 10:
            return "Alta"
        elif intersecao > 5:
            return "Média"
        else:
            return "Baixa"
    
    def relatorio_metricas(self):
        """Gera relatório das métricas"""
        print("📊 RELATÓRIO DE MÉTRICAS RAG")
        print("=" * 40)
        print(f"🔢 Total de consultas: {self.metricas['consultas_realizadas']}")
        print(f"⏱️ Tempo médio de resposta: {self.metricas['tempo_medio_resposta']:.2f}s")
        
        if self.metricas['documentos_encontrados']:
            media_docs = sum(self.metricas['documentos_encontrados']) / len(self.metricas['documentos_encontrados'])
            print(f"📄 Média de documentos encontrados: {media_docs:.1f}")
        
        print("\n📋 Histórico de consultas:")
        for consulta in self.metricas['historico_consultas'][-5:]:  # Últimas 5
            print(f"  {consulta['timestamp']} | {consulta['query']} | "
                  f"Docs: {consulta['documentos_encontrados']} | "
                  f"Tempo: {consulta['tempo_resposta']}s | "
                  f"Qualidade: {consulta['qualidade_estimada']}")

# Criando avaliador
avaliador = AvaliadorRAG()
print("📊 Sistema de avaliação RAG criado!")

In [ ]:
# Testando diferentes consultas com métricas
consultas_teste = [
    "Quais são as principais fintechs brasileiras?",
    "Como está o cenário de startups em São Paulo?",
    "O que você sabe sobre o Porto Digital no Recife?",
    "Conte sobre o Mercado Livre e seu crescimento"
]

print("🧪 Testando sistema RAG com diferentes consultas...\n")

for i, consulta in enumerate(consultas_teste, 1):
    print(f"\n{'='*60}")
    print(f"TESTE {i}: {consulta}")
    print("="*60)
    
    resposta, metricas_consulta = avaliador.avaliar_consulta(
        consulta, agente_tech_brasil, conhecimento_brasil
    )
    
    print(f"\n📊 Métricas desta consulta:")
    print(f"   📄 Documentos encontrados: {metricas_consulta['documentos_encontrados']}")
    print(f"   ⏱️ Tempo de resposta: {metricas_consulta['tempo_resposta']}s")
    
    print(f"\n🤖 Resposta gerada:")
    print(resposta.content[:300] + "..." if len(resposta.content) > 300 else resposta.content)

print("\n" + "="*60)
print("📈 RELATÓRIO FINAL DE MÉTRICAS")
print("="*60)
avaliador.relatorio_metricas()

# 🎯 Exercícios Práticos

Agora é sua vez de colocar a mão na massa!

## 🏋️ Exercício 1: Criando sua própria base de conhecimento

**Desafio:** Crie uma base de conhecimento sobre um tema que você domina (esportes, música, culinária, etc.) e implemente um sistema RAG simples.

**O que fazer:**
1. Escolha um tema
2. Crie 5-10 documentos sobre o tema
3. Implemente busca por similaridade
4. Teste com perguntas específicas

In [ ]:
# SEU CÓDIGO AQUI!
# Dica: Use a estrutura da ConhecimentoTechBrasil como base

class MeuConhecimento:
    def __init__(self):
        # TODO: Adicione sua base de conhecimento aqui
        self.base_conhecimento = {
            # "categoria1": {
            #     "item1": "Descrição detalhada...",
            #     "item2": "Outra descrição..."
            # }
        }
        pass
    
    def buscar_informacao(self, query):
        # TODO: Implemente sua lógica de busca
        pass

# Teste aqui:
# meu_conhecimento = MeuConhecimento()
# resultado = meu_conhecimento.buscar_informacao("sua pergunta")
# print(resultado)

print("💪 Complete este exercício para praticar!")

## 🏋️ Exercício 2: Comparando abordagens RAG

**Desafio:** Compare a mesma pergunta usando:
1. Agente sem conhecimento
2. Agente com RAG tradicional (sempre busca)
3. Agente com Agentic RAG (busca quando necessário)

**Analise:** Qual abordagem dá melhores resultados para diferentes tipos de pergunta?

In [ ]:
# SEU CÓDIGO AQUI!
# Dica: Crie três versões de agentes e teste com a mesma pergunta

def comparar_abordagens_rag(pergunta):
    # TODO: Implemente as três abordagens
    
    # 1. Agente sem conhecimento
    agente_simples = None  # TODO: Criar agente simples
    
    # 2. Agente com RAG tradicional
    agente_rag_tradicional = None  # TODO: Criar com contexto sempre enriquecido
    
    # 3. Agente com Agentic RAG
    agente_agentic_rag = None  # TODO: Criar que decide quando buscar
    
    # TODO: Teste e compare os resultados
    pass

# Teste com diferentes tipos de pergunta:
perguntas_teste = [
    "Qual é a capital do Brasil?",  # Conhecimento geral
    "Como está o preço das ações do Nubank hoje?",  # Informação específica/atualizada
    "Explique como funciona machine learning"  # Conhecimento técnico geral
]

print("🎯 Complete este exercício para entender as diferenças!")

# 🚀 Casos de Uso Avançados

Vamos ver alguns casos de uso mais avançados do sistema RAG!

In [ ]:
# Caso 1: Agente especialista que atualiza seu próprio conhecimento
class AgenteAutoAtualizable:
    def __init__(self, modelo, conhecimento_inicial):
        self.modelo = modelo
        self.conhecimento = conhecimento_inicial
        self.historico_aprendizado = []
    
    def processar_consulta(self, query):
        """Processa consulta e aprende com interações"""
        
        # Busca conhecimento existente
        informacoes = self.conhecimento.buscar_informacao(query)
        
        # Se não encontrou informações suficientes, marca para atualização
        if len(informacoes) < 2:
            self.historico_aprendizado.append({
                'query': query,
                'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                'status': 'conhecimento_insuficiente'
            })
            print(f"⚠️ Conhecimento insuficiente para: '{query}'")
            print("📝 Marcado para futura atualização da base de conhecimento")
        
        # Cria agente especializado
        agente, _ = criar_agente_com_conhecimento(query)
        resposta = agente.run(query)
        
        return resposta
    
    def relatorio_aprendizado(self):
        """Mostra áreas que precisam de mais conhecimento"""
        if not self.historico_aprendizado:
            print("✅ Nenhuma lacuna de conhecimento identificada!")
            return
        
        print("📊 RELATÓRIO DE LACUNAS DE CONHECIMENTO")
        print("=" * 45)
        for item in self.historico_aprendizado:
            print(f"🔍 {item['timestamp']}: {item['query']}")

# Testando agente auto-atualizável
agente_auto = AgenteAutoAtualizable(modelo, conhecimento_brasil)

print("🤖 Testando Agente Auto-Atualizável...\n")

# Testando com consultas variadas
consultas_variadas = [
    "Fale sobre fintechs brasileiras",  # Tem conhecimento
    "Como está o mercado de criptomoedas no Brasil?",  # Pode não ter
    "Quais são as startups de IA no Brasil?",  # Pode não ter
    "Conte sobre o Porto Digital"  # Tem conhecimento
]

for consulta in consultas_variadas:
    print(f"\n❓ Pergunta: {consulta}")
    resposta = agente_auto.processar_consulta(consulta)
    print(f"🤖 Resposta resumida: {resposta.content[:150]}...")

print("\n" + "="*50)
agente_auto.relatorio_aprendizado()

In [ ]:
# Caso 2: Sistema RAG com múltiplas fontes de conhecimento
class SistemaRAGMultiplosFontes:
    def __init__(self):
        self.fontes = {
            'tech_brasil': conhecimento_brasil,
            'mercado_financeiro': self._criar_conhecimento_financeiro(),
            'startups_globais': self._criar_conhecimento_global()
        }
    
    def _criar_conhecimento_financeiro(self):
        class ConhecimentoFinanceiro:
            def buscar_informacao(self, query):
                base = {
                    'investimentos': "No Brasil, principais opções são: Tesouro Direto, CDB, LCI/LCA, ações, fundos imobiliários",
                    'pix': "PIX revolucionou pagamentos no Brasil, com mais de 100 milhões de chaves cadastradas",
                    'bolsa': "B3 é a bolsa de valores brasileira, com mais de 4 milhões de investidores pessoas físicas"
                }
                return [v for k, v in base.items() if k in query.lower()]
        return ConhecimentoFinanceiro()
    
    def _criar_conhecimento_global(self):
        class ConhecimentoGlobal:
            def buscar_informacao(self, query):
                base = {
                    'silicon valley': "Vale do Silício concentra gigantes tech como Google, Apple, Meta, Tesla",
                    'unicornios': "Empresas unicórnio são startups avaliadas em mais de 1 bilhão de dólares",
                    'venture capital': "VC investe em startups early-stage em troca de equity"
                }
                return [v for k, v in base.items() if k in query.lower()]
        return ConhecimentoGlobal()
    
    def buscar_em_todas_fontes(self, query):
        """Busca em todas as fontes de conhecimento"""
        resultados = {}
        
        for nome_fonte, fonte in self.fontes.items():
            infos = fonte.buscar_informacao(query)
            if infos:
                resultados[nome_fonte] = infos
        
        return resultados
    
    def criar_agente_multi_fonte(self, query):
        """Cria agente com conhecimento de múltiplas fontes"""
        todas_infos = self.buscar_em_todas_fontes(query)
        
        contexto_multi = "\n\nCONHECIMENTO DE MÚLTIPLAS FONTES:\n"
        
        for fonte, infos in todas_infos.items():
            contexto_multi += f"\n📚 Fonte: {fonte.upper()}\n"
            for i, info in enumerate(infos, 1):
                contexto_multi += f"  {i}. {info}\n"
        
        agente_multi = Agent(
            name="Especialista Multi-Domínio",
            model=modelo,
            description=f"Você tem acesso a conhecimento de múltiplas fontes especializadas.{contexto_multi}",
            instructions=[
                "Use informações de TODAS as fontes disponíveis",
                "Cite qual fonte você está usando para cada informação",
                "Faça conexões entre informações de diferentes fontes quando relevante"
            ],
            markdown=True
        )
        
        return agente_multi, todas_infos

# Testando sistema multi-fontes
sistema_multi = SistemaRAGMultiplosFontes()

print("🌐 Testando Sistema RAG com Múltiplas Fontes...\n")

query_multi = "Como o PIX impactou as fintechs brasileiras e qual a diferença para sistemas de pagamento globais?"

print(f"❓ Pergunta complexa: {query_multi}\n")

agente_multi, fontes_usadas = sistema_multi.criar_agente_multi_fonte(query_multi)

print("🔍 Fontes de conhecimento consultadas:")
for fonte, infos in fontes_usadas.items():
    print(f"   📚 {fonte}: {len(infos)} informações encontradas")

print("\n🤖 Gerando resposta integrando múltiplas fontes...\n")
resposta_multi = agente_multi.run(query_multi)

print("✨ Resposta integrada:")
print(resposta_multi.content)

print("\n🎯 Vantagem: O agente consegue combinar conhecimento de diferentes domínios!")

# 📊 Visualizando Performance e Métricas

In [ ]:
# Criando visualizações das métricas do sistema RAG
import matplotlib.pyplot as plt
import numpy as np

# Simulando dados de performance
metodos = ['Sem Knowledge', 'RAG Tradicional', 'Agentic RAG', 'Multi-Fonte']
tempo_resposta = [0.8, 2.1, 1.4, 2.8]  # segundos
precisao = [65, 85, 92, 95]  # % de precisão estimada
relevancia = [60, 82, 90, 94]  # % de relevância estimada

# Criando subplot com múltiplos gráficos
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('📊 Análise Comparativa de Sistemas RAG', fontsize=16, fontweight='bold')

# Gráfico 1: Tempo de Resposta
colors1 = ['lightcoral', 'lightblue', 'lightgreen', 'lightyellow']
bars1 = ax1.bar(metodos, tempo_resposta, color=colors1, edgecolor='black')
ax1.set_title('⏱️ Tempo de Resposta (segundos)', fontweight='bold')
ax1.set_ylabel('Segundos')
ax1.tick_params(axis='x', rotation=45)

# Adicionando valores nas barras
for bar, value in zip(bars1, tempo_resposta):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.05, 
             f'{value}s', ha='center', va='bottom', fontweight='bold')

# Gráfico 2: Precisão
bars2 = ax2.bar(metodos, precisao, color=['#ff9999', '#66b3ff', '#99ff99', '#ffcc99'], edgecolor='black')
ax2.set_title('🎯 Precisão das Respostas (%)', fontweight='bold')
ax2.set_ylabel('Precisão (%)')
ax2.set_ylim(0, 100)
ax2.tick_params(axis='x', rotation=45)

for bar, value in zip(bars2, precisao):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
             f'{value}%', ha='center', va='bottom', fontweight='bold')

# Gráfico 3: Relevância
bars3 = ax3.bar(metodos, relevancia, color=['#ffb3ba', '#baffc9', '#bae1ff', '#ffffba'], edgecolor='black')
ax3.set_title('📈 Relevância do Conteúdo (%)', fontweight='bold')
ax3.set_ylabel('Relevância (%)')
ax3.set_ylim(0, 100)
ax3.tick_params(axis='x', rotation=45)

for bar, value in zip(bars3, relevancia):
    ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
             f'{value}%', ha='center', va='bottom', fontweight='bold')

# Gráfico 4: Radar Chart comparativo
categorias = ['Velocidade', 'Precisão', 'Relevância', 'Facilidade']
valores_agentic = [85, 92, 90, 95]  # Valores normalizados para Agentic RAG

# Calculando ângulos para o radar
angulos = np.linspace(0, 2 * np.pi, len(categorias), endpoint=False).tolist()
valores_agentic += valores_agentic[:1]  # Fechando o círculo
angulos += angulos[:1]

ax4.plot(angulos, valores_agentic, 'o-', linewidth=2, label='Agentic RAG', color='green')
ax4.fill(angulos, valores_agentic, alpha=0.25, color='green')
ax4.set_xticks(angulos[:-1])
ax4.set_xticklabels(categorias)
ax4.set_ylim(0, 100)
ax4.set_title('🕸️ Perfil Agentic RAG', fontweight='bold')
ax4.grid(True)

plt.tight_layout()
plt.show()

print("📊 Análise dos Resultados:")
print("\n🏆 VENCEDORES:")
print("   ⚡ Velocidade: Sem Knowledge (mais simples)")
print("   🎯 Precisão: Multi-Fonte (mais completo)")
print("   ⚖️ Equilíbrio: Agentic RAG (melhor custo-benefício)")
print("\n💡 Conclusão: Agentic RAG oferece o melhor equilíbrio!")

# 🎉 Resumo e Próximos Passos

## O que aprendemos hoje:

### 🧠 Conceitos Fundamentais:
- **Knowledge**: Como dar conhecimento especializado aos agentes
- **Embeddings**: Representação matemática de texto para busca semântica
- **Vector Databases**: Armazenamento otimizado para busca por similaridade
- **Agentic RAG**: Agentes que decidem quando buscar conhecimento

### 🛠️ Implementações Práticas:
- Sistema RAG básico com busca por similaridade
- Agentes com conhecimento enriquecido
- Métricas e avaliação de performance
- Sistemas multi-fonte para conhecimento abrangente

### 📊 Principais Diferenças:
- **RAG Tradicional**: Sempre busca na base (mais lento, mas completo)
- **Agentic RAG**: Busca apenas quando necessário (mais eficiente)
- **Multi-Fonte**: Combina diferentes domínios de conhecimento

## 🚀 Próximos Módulos:
- **Módulo 5**: Memória e Persistência - Como fazer agentes lembrarem de conversas anteriores
- **Módulo 6**: Agentes Multimodal - Processamento de imagens e outros tipos de mídia
- **Módulo 7**: Reasoning - Como fazer agentes "pensarem" antes de agir

![](https://s3.us-east-1.amazonaws.com/turing.education/books/imagens/agno-2.0-framework--modulo-04_img_02.png)

### 💡 Dica Final:
O **Knowledge** é o que transforma seus agentes de "inteligentes" para "especialistas"! Use essa ferramenta para criar sistemas que realmente entendem do seu domínio específico.

**Liiindo!** Agora seus agentes têm cérebros especializados! 🧠✨

# 🎯 Desafio Final

Para consolidar seu aprendizado, implemente um sistema RAG completo para um domínio de sua escolha:

## 📋 Requisitos:
1. ✅ Base de conhecimento com pelo menos 10 documentos
2. ✅ Sistema de busca por similaridade
3. ✅ Agente especializado com Agentic RAG
4. ✅ Métricas de avaliação (tempo, precisão, relevância)
5. ✅ Teste com pelo menos 5 perguntas diferentes

## 🏆 Bonus:
- Implemente múltiplas fontes de conhecimento
- Adicione sistema de auto-avaliação
- Crie visualizações das métricas

**Boa sorte e bora codar!** 🚀